In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import time
import numpy as np
import tensorflow as tf
from VGG16_flatten import VGG16

In [2]:
import numpy as np
import pandas as pd
import skimage.io as imageio
import pickle

In [3]:
from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

In [4]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [5]:
HOME_DIR = "/home/cmchang/DLCV2018SPRING/final/"
TRAIN_DIR = HOME_DIR+"dlcv_final_2_dataset/train/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

In [6]:
dtrain = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/train_id.txt", header=None,sep=" ", names=["img", "id"])
dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

In [7]:
train_list = list(TRAIN_DIR+dtrain.img)
valid_list = list(VALID_DIR+dvalid.img)

In [8]:
def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

In [9]:
def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

In [10]:
def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [11]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [12]:
Xtrain = readImgList(train_list)

In [13]:
Xvalid = readImgList(valid_list)

In [14]:
Xtrain.shape

(56475, 218, 178, 3)

In [15]:
ytrain = transformLabel(list(dtrain.id), y_dict)

In [16]:
yvalid = transformLabel(list(dvalid.id), y_dict)

In [17]:
Ytrain = one_hot_encoding(ytrain, len(y_dict))
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [18]:
scope_name = "Model"

In [19]:
model = VGG16(scope_name=scope_name)

In [20]:
FLAG_init_from = "updated_keras_vgg16.npy"
FLAG_prof_type = "all-one"
FLAG_lambda_s = 2e-7
FLAG_lambda_m = 2e-7
FLAG_decay = 1e-5
FLAG_lr = 4e-4
FLAG_keep_prob = 1.0
FLAG_save_dir = HOME_DIR+"rescale0-1_v2_save_all-one/"

In [21]:
model.build(vgg16_npy_path=FLAG_init_from,
            shape=Xtrain.shape[1:],
            classes=len(y_dict),
            prof_type=FLAG_prof_type,
            conv_pre_training=False,
            fc_pre_training=False)

build model started
npy file loaded
build model finished: 1s


In [22]:
model.sparsity_train(l1_gamma=FLAG_lambda_s, l1_gamma_diff=FLAG_lambda_m, decay=FLAG_decay, keep_prob=FLAG_keep_prob)

[None, 218, 178, 3]
[None, 218, 178, 64]
AFTER [None, 218, 178, 64]
[None, 109, 89, 64]
AFTER [None, 109, 89, 64]
[None, 109, 89, 128]
AFTER [None, 109, 89, 128]
[None, 55, 45, 128]
AFTER [None, 55, 45, 128]
[None, 55, 45, 256]
AFTER [None, 55, 45, 256]
[None, 55, 45, 256]
AFTER [None, 55, 45, 256]
[None, 28, 23, 256]
AFTER [None, 28, 23, 256]
[None, 28, 23, 512]
AFTER [None, 28, 23, 512]
[None, 28, 23, 512]
AFTER [None, 28, 23, 512]
[None, 14, 12, 512]
AFTER [None, 14, 12, 512]
[None, 14, 12, 512]
AFTER [None, 14, 12, 512]
[None, 14, 12, 512]
AFTER [None, 14, 12, 512]
sparsity train operation setup: 1s


In [23]:
import imgaug as ia
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
transform = iaa.Sequential([
    sometimes(iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})),
    sometimes(iaa.Affine(scale={"x": (0.85, 1.15), "y":(0.85, 1.15)})),
    sometimes(iaa.Affine(rotate=(-45, 45))),
    sometimes(iaa.Fliplr(0.5))
])

In [24]:
print("===== create directory =====")
if not os.path.exists(FLAG_save_dir):
    os.makedirs(FLAG_save_dir)

arr_spareness = []

# define tasks
tasks = ['var_dp']
print(tasks)

# initial task
cur_task = tasks[0]
obj = model.loss_dict[tasks[0]]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

   # hyper parameters
    batch_size = 64
    epoch = 100
    early_stop_patience = 10
    min_delta = 0.0001
    opt_type = 'adam'

    # recorder
    epoch_counter = 0
    history = list()

    # optimizer
    global_step = tf.Variable(0, trainable=False)

    # Passing global_step to minimize() will increment it at each step.
    learning_rate = FLAG_lr # adam # 4e-3 #sgd
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)

    checkpoint_path = os.path.join(FLAG_save_dir, 'model.ckpt')
    tvars_trainable = tf.trainable_variables()

    train_vars = list()
    for var in tf.trainable_variables():
        if model.scope_name in var.name:
            train_vars.append(var)
            
    train_op = opt.minimize(obj, global_step=global_step, var_list=tvars_trainable)
    
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=len(tasks))

    # progress bar
    ptrain = IntProgress()
    pval = IntProgress()
    display(ptrain)
    display(pval)
    ptrain.max = int(Xtrain.shape[0]/batch_size)
    pval.max = int(Xvalid.shape[0]/batch_size)

    spareness = model.spareness(thresh=0.05)
    print("initial spareness: %s" % sess.run(spareness))

    # re-initialize
    initialize_uninitialized(sess)

    # reset due to adding a new task
    patience_counter = 0
    current_best_val_accu = 0

    # optimize when the aggregated obj
    while(patience_counter < early_stop_patience and epoch_counter < epoch):

        def load_batches():
            for i in range(int(Xtrain.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                batch = ia.Batch(images=Xtrain[st:ed,:,:,:], data=Ytrain[st:ed,:])
                yield batch

        batch_loader = ia.BatchLoader(load_batches)
        bg_augmenter = ia.BackgroundAugmenter(batch_loader=batch_loader, augseq=transform, nb_workers=8)

        # start training
        stime = time.time()
        bar_train = Bar('Training', max=int(Xtrain.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        bar_val =  Bar('Validation', max=int(Xvalid.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        train_loss, train_accu = 0.0, 0.0
        while True:
            batch = bg_augmenter.get_batch()
            if batch is None:
                print("Finished epoch.")
                break
            x_images_aug = batch.images_aug
            y_images = batch.data
            loss, accu, _ = sess.run([obj, model.accu_dict[cur_task], train_op], feed_dict={model.x: x_images_aug,
                            model.y: y_images,
                            model.is_train: True})
            bar_train.next()
            train_loss += loss
            train_accu += accu
            ptrain.value +=1
            ptrain.description = "Training %s/%s" % (ptrain.value, ptrain.max)
        train_loss = train_loss/ptrain.value
        train_accu = train_accu/ptrain.value
        batch_loader.terminate()
        bg_augmenter.terminate()

        # # training an epoch
        # for i in range(int(Xtrain.shape[0]/batch_size)):
        #     st = i*batch_size
        #     ed = (i+1)*batch_size

        #     augX = transform.augment_images(Xtrain[st:ed,:,:,:])

        #     sess.run([train_op], feed_dict={model.x: augX,
        #                                     model.y: Ytrain[st:ed,:],
        #                                     model.is_train: False})
        #     ptrain.value +=1
        #     ptrain.description = "Training %s/%s" % (i, ptrain.max)
        #     bar_train.next()

        # validation
        val_loss = 0
        val_accu = 0
        for i in range(int(Xvalid.shape[0]/batch_size)):
            st = i*batch_size
            ed = (i+1)*batch_size
            loss, accu = sess.run([obj, model.accu_dict[cur_task]],
                                feed_dict={model.x: Xvalid[st:ed,:],
                                            model.y: Yvalid[st:ed,:],
                                            model.is_train: False})
            val_loss += loss
            val_accu += accu
            pval.value += 1
            pval.description = "Testing %s/%s" % (pval.value, pval.value)
        val_loss = val_loss/pval.value
        val_accu = val_accu/pval.value

        print("\nspareness: %s" % sess.run(spareness))
        # early stopping check
        if (val_accu - current_best_val_accu) > min_delta:
            current_best_val_accu = val_accu
            patience_counter = 0

            para_dict = sess.run(model.para_dict)
            np.save(os.path.join(FLAG_save_dir, "para_dict.npy"), para_dict)
            print("save in %s" % os.path.join(FLAG_save_dir, "para_dict.npy"))
        else:
            patience_counter += 1

        # shuffle Xtrain and Ytrain in the next epoch
        idx = np.random.permutation(Xtrain.shape[0])
        Xtrain, Ytrain = Xtrain[idx,:,:,:], Ytrain[idx,:]

        # epoch end
        # writer.add_summary(epoch_summary, epoch_counter)
        epoch_counter += 1

        ptrain.value = 0
        pval.value = 0
        bar_train.finish()
        bar_val.finish()

        print("Epoch %s (%s), %s sec >> train loss: %.4f, train accu: %.4f, val loss: %.4f, val accu at %s: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), train_loss, train_accu, val_loss, cur_task, val_accu))
        history.append([train_loss, train_accu, val_loss, val_accu ])
        
        if epoch_counter % 10 == 0:
            import matplotlib.pyplot as plt
            df = pd.DataFrame(history)
            df.columns = ['train_loss', 'train_accu', 'val_loss', 'val_accu']
            df[['train_loss', 'val_loss']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'loss.png'))
            df[['train_accu', 'val_accu']].plot()
            plt.savefig(os.path.join(FLAG_save_dir, 'accu.png'))
            
    saver.save(sess, checkpoint_path, global_step=epoch_counter)

    #sp, rcut = gammaSparsifyVGG16(para_dict, thresh=0.02)
    #np.save(os.path.join(FLAG_save_dir,"sparse_dict.npy"), sp)
    #print("sparsify %s in %s" % (np.round(1-rcut,3), os.path.join(FLAG_save_dir, "sparse_dict.npy")))

    #writer.close()
    #arr_spareness.append(1-rcut)
    #np.save(os.path.join(FLAG_save_dir,"sprocess.npy"), arr_spareness)
# FLAG_optimizer = opt_type
# FLAG_lr = start_learning_rate
# FLAG_batch_size = batch_size
# FLAG_epoch_end = epoch_counter
# FLAG_val_accu = current_best_val_accu

# header = ''
# row = ''
# for key in sorted(vars(FLAG)):
#     if header is '':
#         header = key
#         row = str(getattr(FLAG, key))
#     else:
#         header += ","+key
#         row += ","+str(getattr(FLAG,key))
# row += "\n"
# header += "\n"
# if os.path.exists("/home/cmchang/new_CP_CNN/model.csv"):
#     with open("/home/cmchang/new_CP_CNN/model.csv", "a") as myfile:
#         myfile.write(row)
# else:
#     with open("/home/cmchang/new_CP_CNN/model.csv", "w") as myfile:
#         myfile.write(header)
#         myfile.write(row)

===== create directory =====
['var_dp']


IntProgress(value=0)

IntProgress(value=0)

initial spareness: 0.0
Finished epoch.

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/rescale0-1_v2_save_all-one/para_dict.npy
Epoch 1 (0), 891.2 sec >> train loss: 9.8467, train accu: 0.0015, val loss: 13.4347, val accu at var_dp: 0.0004
Finished epoch.

spareness: 0.0
Epoch 2 (1), 842.62 sec >> train loss: 9.1615, train accu: 0.0005, val loss: 9.1121, val accu at var_dp: 0.0000
Finished epoch.

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/rescale0-1_v2_save_all-one/para_dict.npy
Epoch 3 (0), 847.87 sec >> train loss: 9.0509, train accu: 0.0005, val loss: 8.9923, val accu at var_dp: 0.0010
Finished epoch.

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/rescale0-1_v2_save_all-one/para_dict.npy
Epoch 4 (0), 847.33 sec >> train loss: 8.8815, train accu: 0.0011, val loss: 8.7208, val accu at var_dp: 0.0018
Finished epoch.

spareness: 0.0
save in /home/cmchang/DLCV2018SPRING/final/rescale0-1_v2_save_all-one/para_dict.npy
Epoch 5 (0), 849.97 sec >> train 

Finished epoch.

spareness: 0.00023674243
Epoch 44 (9), 849.25 sec >> train loss: 0.9200, train accu: 0.9423, val loss: 4.3818, val accu at var_dp: 0.5197
Finished epoch.

spareness: 0.00023674243
Epoch 45 (10), 850.97 sec >> train loss: 0.9088, train accu: 0.9419, val loss: 4.4647, val accu at var_dp: 0.5215
